In [ ]:
import os
import torch
from PIL import Image
from tqdm.auto import tqdm, trange
from metrics import *
from dev import *

# Example paths
clean_path = "/fs/nexus-projects/HuangWM/datasets/main/diffusiondb/real"
attacked_path = "/fs/nexus-projects/HuangWM/datasets/attacked/diffusiondb/distortion_single_rotation-9-real"

# Loading all, for easy testing
clean_images = [
    Image.open(os.path.join(clean_path, f"{idx}.png")) for idx in range(LIMIT)
]
attacked_images = [
    Image.open(os.path.join(attacked_path, f"{idx}.png")) for idx in range(LIMIT)
]

In [ ]:
compute_fid(
    clean_path,
    attacked_path,
    mode="legacy",
    device=torch.device("cuda"),
    batch_size=128,
    num_workers=8,
    verbose=True,
)

compute_fid(
    clean_path,
    attacked_path,
    mode="clip",
    device=torch.device("cuda"),
    batch_size=128,
    num_workers=8,
    verbose=True,
)

In [ ]:
compute_psnr_repeated(clean_images, attacked_images, num_workers=8, verbose=True)
compute_ssim_repeated(clean_images, attacked_images, num_workers=8, verbose=True)
compute_nmi_repeated(clean_images, attacked_images, num_workers=8, verbose=True)

In [ ]:
model = load_perceptual_models("lpips", mode="alex", device=torch.device("cuda"))

batch_size = 32
for i in trange(0, len(clean_images), batch_size):
    compute_lpips_repeated(
        clean_images[i : min(i + batch_size, len(clean_images))],
        attacked_images[i : min(i + batch_size, len(clean_images))],
        mode="alex",
        model=model,
        device=torch.device("cuda"),
    )

In [ ]:
model = load_perceptual_models("watson", mode="dft", device=torch.device("cuda"))


batch_size = 32
for i in trange(0, len(clean_images), batch_size):
    compute_watson_repeated(
        clean_images[i : min(i + batch_size, len(clean_images))],
        attacked_images[i : min(i + batch_size, len(clean_images))],
        mode="dft",
        model=model,
        device=torch.device("cuda"),
    )

In [ ]:
model = load_aesthetics_and_artifacts_models(device=torch.device("cuda"))

batch_size = 32
for i in trange(0, len(clean_images), batch_size):
    aesthetics_score, artifaces_score = compute_aesthetics_and_artifacts_scores(
        clean_images[i : min(i + batch_size, len(clean_images))],
        model,
        device=torch.device("cuda"),
    )
    print(aesthetics_score, artifaces_score)

In [ ]:
models = load_open_clip_model_preprocess_and_tokenizer(device=torch.device("cuda"))
prompts = ["A photo of a cat"] * len(clean_images)

batch_size = 32
for i in trange(0, len(clean_images), batch_size):
    clip_scores = compute_clip_score(
        clean_images[i : min(i + batch_size, len(clean_images))],
        prompts[i : min(i + batch_size, len(clean_images))],
        models,
        device=torch.device("cuda"),
    )
    print(clip_scores)